In [ ]:
import pandas as pd

# Load the dataset from a CSV file
df = pd.read_csv("dataset.csv")

# Combine 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single 'timestamp' column
# Format is specified to ensure correct datetime parsing
df['timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Map traffic condition levels from categorical to numerical values for modeling
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Convert the 'VehicleType' column into one-hot encoded dummy variables
vehicle_dummies = pd.get_dummies(df['VehicleType'], prefix='Vehicle')

# Append the dummy variables to the original DataFrame
df = pd.concat([df, vehicle_dummies], axis=1)

# Drop columns that have been processed and are no longer needed
df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime', 'TrafficConditionNearby', 'VehicleType'], inplace=True)

# Check for missing values in the dataset and print the count per column
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

# Remove duplicate rows from the dataset, if any
df.drop_duplicates(inplace=True)

# Sort the DataFrame by parking lot and timestamp to maintain chronological order
df.sort_values(by=['SystemCodeNumber', 'timestamp'], inplace=True)

# Reset the index to reflect the new ordering after sorting
df.reset_index(drop=True, inplace=True)

# Display the first few rows of the processed DataFrame for quick inspection
print(df.head())



In [ ]:
import pandas as pd

# Load the raw dataset
df = pd.read_csv("dataset.csv")

# Combine and convert date and time columns into a single ISO-formatted timestamp
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')  # Format to ISO 8601

# Map traffic conditions to numerical values
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['TrafficLevel'] = df['TrafficLevel'].fillna(0).astype(int)  # Replace missing with 0

# Convert 'VehicleType' categorical column into dummy variables
vehicle_dummies = pd.get_dummies(df['VehicleType'], prefix='Vehicle')
df = pd.concat([df, vehicle_dummies], axis=1)

# Ensure all expected dummy columns are present, even if missing in input
for col in ['Vehicle_car', 'Vehicle_bike', 'Vehicle_truck']:
    if col not in df:
        df[col] = 0

# Drop raw categorical and now redundant columns
df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime', 'TrafficConditionNearby', 'VehicleType'], inplace=True)

# Fill missing values in integer columns with 0 and ensure correct type
int_columns = ['Capacity', 'Occupancy', 'QueueLength', 'IsSpecialDay', 'TrafficLevel',
               'Vehicle_car', 'Vehicle_bike', 'Vehicle_truck']
df[int_columns] = df[int_columns].fillna(0).astype(int)

# Remove any duplicate rows
df.drop_duplicates(inplace=True)

# Sort the dataset chronologically within each parking lot
df.sort_values(by=['SystemCodeNumber', 'timestamp'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Save the cleaned and formatted dataset for use in dynamic pricing models
df.to_csv("parking_stream.csv", index=False)

# Display sample of the cleaned dataset
print("Cleaned data ready for streaming:\n")
print(df.head())


Urban Parking Dynamic Pricing – Model 1 (Baseline) Using Pathway

This script simulates real-time dynamic pricing for urban parking using a baseline occupancy-based model,
executed via the Pathway real-time data processing engine.

The baseline pricing logic increases the parking fee linearly with occupancy using the formula:
    price = BASE_PRICE + ALPHA × (occupancy / capacity)

Key Steps:
1. Preprocess the raw parking dataset and encode features.
2. Simulate a streaming environment using Pathway with a subset of data.
3. Define a simple pricing function based on occupancy ratio.
4. Stream processed data through the model and output results to CSV.

In [ ]:
import pandas as pd
import pathway as pw
from pathway.internals.dtype import DATE_TIME_NAIVE
from datetime import datetime

# --- STEP 1: Data Preprocessing ---

# Load full raw dataset
df = pd.read_csv("dataset.csv")

# Combine date and time columns into ISO timestamp format
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Map traffic condition to numeric levels (low=1, medium=2, high=3)
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['TrafficLevel'] = df['TrafficLevel'].fillna(0).astype(int)

# One-hot encode 'VehicleType' into separate binary columns
vehicle_dummies = pd.get_dummies(df['VehicleType'], prefix='Vehicle')
df = pd.concat([df, vehicle_dummies], axis=1)

# Ensure consistent schema by adding missing dummy columns if needed
for col in ['Vehicle_car', 'Vehicle_bike', 'Vehicle_truck']:
    if col not in df:
        df[col] = 0

# Drop columns no longer required after encoding and timestamp creation
df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime', 'TrafficConditionNearby', 'VehicleType'], inplace=True)

# Convert all necessary columns to integers, filling any missing values with 0
int_cols = ['Capacity', 'Occupancy', 'QueueLength', 'IsSpecialDay', 'TrafficLevel',
            'Vehicle_car', 'Vehicle_bike', 'Vehicle_truck']
df[int_cols] = df[int_cols].fillna(0).astype(int)

# Limit data to 1 parking lot (first in the list) and first 200 records to simulate streaming
lot = df['SystemCodeNumber'].unique()[0]
df_small = df[df['SystemCodeNumber'] == lot].head(200).copy()

# Save this subset for use in Pathway streaming
df_small.to_csv("parking_stream.csv", index=False)
print(f"Saved subset for {lot} with {len(df_small)} rows.")

# --- STEP 2: Define Pathway Schema ---
# Schema defines data structure for the stream
class ParkingStream(pw.Schema):
    timestamp: DATE_TIME_NAIVE
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    Vehicle_bike: int
    Vehicle_car: int
    Vehicle_truck: int

# --- STEP 3: Read Streaming Data ---
# Load the CSV as a static stream (change to "streaming" for live input)
stream = pw.io.csv.read(
    "parking_stream.csv",
    schema=ParkingStream,
    # mode="streaming",
    # autocommit_duration_ms=10  # Optional: controls commit frequency in streaming mode
)

# --- STEP 4: Baseline Pricing Model ---
# Define constants for pricing formula
BASE_PRICE = 10.0
ALPHA = 2.0
MIN_PRICE = 5.0
MAX_PRICE = 20.0

# Define user-defined function for baseline pricing based on occupancy ratio
@pw.udf
def baseline_price_fn(occupancy, capacity):
    return max(MIN_PRICE, min(MAX_PRICE, BASE_PRICE + ALPHA * (occupancy / capacity)))

# --- STEP 5: Apply Pricing Formula ---
# Apply pricing logic and extract relevant fields
pricing = stream.select(
    timestamp=stream.timestamp,
    lot=stream.SystemCodeNumber,
    price=baseline_price_fn(stream.Occupancy, stream.Capacity)
)

# --- STEP 6: Export Result ---
# Write the result to an output CSV
pw.io.csv.write(pricing, "baseline_output.csv")

# --- STEP 7: Run Pathway Application ---
pw.run()


In [ ]:
# Import required modules from Bokeh for interactive plotting
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
import pandas as pd

# --- Step 1: Load and preprocess the dataset ---

# Load the raw dataset from CSV
df = pd.read_csv("dataset.csv")

# Combine the date and time columns into a single datetime object
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Drop the original date and time columns as they're now redundant
df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime'], inplace=True)

# --- Step 2: Define baseline model parameters ---

BASE_PRICE = 10.0   # Starting base price
ALPHA = 2.0         # Sensitivity to occupancy
MIN_PRICE = 5.0     # Minimum allowed price
MAX_PRICE = 20.0    # Maximum allowed price

# --- Step 3: Define a function to compute dynamic price per lot ---

def compute_price(data):
    """
    Applies the baseline dynamic pricing model to a single lot's data,
    adjusting price based on occupancy over time.
    """
    data = data.sort_values(by='timestamp').copy()
    data['Price'] = BASE_PRICE  # Start with base price

    for i in range(1, len(data)):
        prev = data.iloc[i-1]  # Previous row
        occ = data.iloc[i]['Occupancy']
        cap = data.iloc[i]['Capacity']
        # Update price based on previous value and occupancy ratio
        data.at[data.index[i], 'Price'] = max(MIN_PRICE, min(MAX_PRICE, prev['Price'] + ALPHA * (occ / cap)))

    return data

# --- Step 4: Prepare data for all parking lots ---

# Get list of all unique lot IDs
lots = df['SystemCodeNumber'].unique()

# Compute price data for each lot
lot_data = {lot: compute_price(df[df['SystemCodeNumber'] == lot]) for lot in lots}

# Convert each lot's data to a Bokeh-compatible ColumnDataSource
sources = {lot: ColumnDataSource(lot_data[lot]) for lot in lots}

# Select the first lot as the default view
default_lot = lots[0]
source = sources[default_lot]

# --- Step 5: Create the Bokeh plot figure ---

p = figure(
    x_axis_type='datetime',
    title=f"Dynamic Pricing - {default_lot}",
    width=900,
    height=400
)

# Plot price over time for the default lot
p.line(x='timestamp', y='Price', source=source, line_width=2)

# --- Step 6: Create dropdown widget for lot selection ---

lot_selector = Select(title="Select Parking Lot:", value=default_lot, options=list(lots))

# Callback function that updates the plot when dropdown value changes
def update_plot(attr, old, new):
    lot = lot_selector.value  # Get selected lot
    source.data = sources[lot].data  # Update data source
    p.title.text = f"Dynamic Pricing - {lot}"  # Update plot title

# Link dropdown changes to the callback function
lot_selector.on_change('value', update_plot)

# --- Step 7: Set up layout and run app ---

layout = column(lot_selector, p)  # Stack dropdown and plot vertically
curdoc().add_root(layout)         # Add layout to the document
curdoc().title = "Model 1 Pricing"  # Set the title of the Bokeh app



# MODEL 2:
Demand-Based Pricing using Pathway
This script reads a batch of parking data from CSV,
applies a demand-based dynamic pricing formula,
and saves the output prices to a CSV file.


In [ ]:
# Import necessary libraries
import pandas as pd
import pathway as pw
from pathway.internals.dtype import DATE_TIME_NAIVE
from datetime import datetime

# Step 1: Data is assumed to be preprocessed and saved as 'parking_stream.csv'
# (See earlier preprocessing script for steps to generate this file)

# Step 2: Define Schema for the streaming/batch data
# This schema outlines the structure of the data we expect from the CSV
class ParkingStream(pw.Schema):
    timestamp: DATE_TIME_NAIVE        # Cleaned timestamp
    SystemCodeNumber: str             # Parking lot identifier
    Capacity: int                     # Total capacity of the lot
    Occupancy: int                    # Current occupancy
    QueueLength: int                  # Number of vehicles in queue
    IsSpecialDay: int                 # Whether it's a special day (1/0)
    TrafficLevel: int                 # Encoded traffic condition nearby
    Vehicle_bike: int                 # Dummy variable (from one-hot encoding)
    Vehicle_car: int                  # Dummy variable (from one-hot encoding)
    Vehicle_truck: int                # Dummy variable (from one-hot encoding)

# Step 3: Read the CSV data in **batch mode**
# Not using real-time streaming here, just processing one batch
stream = pw.io.csv.read(
    "parking_stream.csv",            # Preprocessed input file
    schema=ParkingStream             # Use the schema defined above
)

# Step 4: Define demand-based dynamic pricing logic
# We use several features to determine price dynamically
BASE_PRICE = 10.0                    # Base starting price
ALPHA = 2.0                          # Factor for occupancy effect
BETA = 1.5                           # Factor for queue length effect
GAMMA = 0.5                          # Factor for traffic level
DELTA = 1.0                          # Add-on if it's a special day
MIN_PRICE = 5.0                      # Price floor
MAX_PRICE = 30.0                     # Price ceiling

# Define user-defined function (UDF) to compute price
@pw.udf
def demand_price_fn(occ, cap, qlen, traffic, special):
    # Compute raw price using weighted influence of features
    raw_price = (
        BASE_PRICE
        + ALPHA * (occ / cap)
        + BETA * (qlen / cap)
        + GAMMA * traffic
        + DELTA * special
    )
    # Clip the price to stay within bounds
    return max(MIN_PRICE, min(MAX_PRICE, raw_price))

# Step 5: Apply the demand-based model to each record
pricing = stream.select(
    timestamp=stream.timestamp,
    lot=stream.SystemCodeNumber,
    price=demand_price_fn(
        stream.Occupancy,
        stream.Capacity,
        stream.QueueLength,
        stream.TrafficLevel,
        stream.IsSpecialDay
    )
)

# Step 6: Output results to CSV
# This is a one-time output for the batch data (not continuously updated)
pw.io.csv.write(pricing, "model2_demand_output.csv")

# Step 7: Run the Pathway pipeline to execute the transformations
pw.run()



# Model Comparison Plot with Annotations
This script compares Model 1 (Baseline) and Model 2 (Demand-Based)
by plotting their pricing outputs over time for a single parking lot.
It also highlights the top 3 highest prices from Model 2.


In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Load pricing outputs from both models
df_model1 = pd.read_csv("baseline_output.csv")
df_model2 = pd.read_csv("model2_demand_output.csv")

# Convert 'timestamp' column to datetime format for proper plotting
df_model1["timestamp"] = pd.to_datetime(df_model1["timestamp"])
df_model2["timestamp"] = pd.to_datetime(df_model2["timestamp"])

# Filter for a single parking lot (ensures fair comparison)
lot = "BHMBCCMKT01"
df_model1 = df_model1[df_model1["lot"] == lot].sort_values("timestamp")
df_model2 = df_model2[df_model2["lot"] == lot].sort_values("timestamp")

# Identify top 3 highest price points in Model 2 for annotation
top_prices = df_model2.sort_values("price", ascending=False).head(3)

# Create a line plot comparing prices from both models
plt.figure(figsize=(14, 6))
plt.plot(df_model1["timestamp"], df_model1["price"], label="Model 1 (Baseline)", marker="o")
plt.plot(df_model2["timestamp"], df_model2["price"], label="Model 2 (Demand-Based)", marker="x")

# Annotate top 3 price spikes in Model 2
for idx, row in top_prices.iterrows():
    plt.annotate(
        f"${row['price']:.2f}",                # Text to display
        (row["timestamp"], row["price"]),      # Location of the point
        textcoords="offset points",            # Position relative to point
        xytext=(0, 10),                        # Offset above the point
        ha='center',                           # Text alignment
        fontsize=9,
        color='red',
        arrowprops=dict(arrowstyle='->', color='gray')  # Arrow pointing to the point
    )

# Final plot formatting
plt.title(f"Model Comparison with Annotations for {lot}")
plt.xlabel("Time")
plt.ylabel("Price ($)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()          # Display the plot



# Model 3: Geo + Demand-Based Dynamic Pricing Model
This script estimates dynamic parking prices by
incorporating both local lot occupancy and neighboring lot demand.

In [ ]:
# Import necessary libraries
import pandas as pd
import pathway as pw
from geopy.distance import geodesic  # Used to compute distance between GPS coordinates

# --- STEP 1: Define a user-defined function to compute geographic distance (in kilometers)
@pw.udf
def geo_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    return geodesic((lat1, lon1), (lat2, lon2)).km

# --- STEP 2: Define the input schema structure for the parking stream
class ParkingStream(pw.Schema):
    timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    Latitude: float
    Longitude: float

# --- STEP 3: Read a static stream (CSV input) using Pathway
stream = pw.io.csv.read("model3_sample.csv", schema=ParkingStream, mode="static")

# --- STEP 4: Create a self-join of the dataset to compare each lot to all others
# This allows calculating pairwise distances between all parking lots
stream_copy = stream.copy()
raw_neighbors = stream.join(stream_copy, how=pw.JoinMode.INNER).select(
    lot=stream.SystemCodeNumber,
    neighbor=stream_copy.SystemCodeNumber,
    dist=geo_distance(  # Calculate distance between original and copied stream records
        stream.Latitude, stream.Longitude,
        stream_copy.Latitude, stream_copy.Longitude
    ),
    occ=stream_copy.Occupancy,  # Occupancy of neighbor lot
    cap=stream_copy.Capacity    # Capacity of neighbor lot
)

# --- STEP 5: Filter out the same lot (self-joins) and keep only neighbors within 1km
neighbors = raw_neighbors.filter(
    (raw_neighbors.lot != raw_neighbors.neighbor) & (raw_neighbors.dist < 1.0)
)

# --- STEP 6: (Optional) Save neighbors to CSV for debugging
pw.io.csv.write(neighbors, "model3_neighbors_debug.csv")

# --- STEP 7: Aggregate demand from neighboring lots
# Computes the sum of occupancy ratios of nearby lots per lot
neighbor_demand = neighbors.groupby(neighbors.lot).reduce(
    neighbors.lot,
    nearby_demand=pw.reducers.sum(neighbors.occ / neighbors.cap)
)

# (Optional) Save neighbor demand table for debugging
pw.io.csv.write(neighbor_demand, "model3_neighbor_demand_debug.csv")

# --- STEP 8: Define dynamic pricing function based on:
# - Current occupancy
# - Capacity
# - Total nearby demand (from nearby lots)
BASE_PRICE = 5.0

@pw.udf
def dynamic_price_fn(occ: int, cap: int, nearby_demand: float) -> float:
    if cap == 0:
        return BASE_PRICE  # Avoid division by zero
    occ_ratio = occ / cap
    # Final price = base adjusted by own occupancy and half of nearby demand
    return round(BASE_PRICE * (1 + occ_ratio + 0.5 * nearby_demand), 2)

# --- STEP 9: Join stream with neighbor demand based on SystemCodeNumber
joined = stream.join(neighbor_demand, stream.SystemCodeNumber == neighbor_demand.lot, how=pw.JoinMode.LEFT)

# --- STEP 10: Apply pricing function, replacing missing neighbor demand with 0
enriched = joined.select(
    timestamp=joined.timestamp,
    lot=joined.SystemCodeNumber,
    price=dynamic_price_fn(
        joined.Occupancy,
        joined.Capacity,
        pw.coalesce(joined.nearby_demand, 0.0)  # Use 0 if no neighbors found
    )
)

# --- STEP 11: Format final output columns
final_output = enriched.select(
    timestamp=enriched.timestamp,
    SystemCodeNumber=enriched.lot,
    price=enriched.price
)

# --- Optional Debugging Outputs: Save the keys used in join for validation
pw.io.csv.write(stream.select(code=stream.SystemCodeNumber), "debug_keys_stream.csv")
pw.io.csv.write(neighbor_demand.select(code=neighbor_demand.lot), "debug_keys_neighbors.csv")

# --- STEP 12: Write final output of Model 3 to CSV
pw.io.csv.write(final_output, "model3_output.csv")

# --- STEP 13: Execute the Pathway data pipeline
pw.run()



#Compare Baseline Model (Model 1) with Model 3
This script merges the pricing outputs of both models
and computes the difference in predicted prices


In [ ]:
import pandas as pd

# Load pricing outputs from both models
baseline_df = pd.read_csv("baseline_output.csv")   # Output from Model 1
model3_df = pd.read_csv("model3_output.csv")       # Output from Model 3

# Rename 'lot' column to 'SystemCodeNumber' in baseline_df
# to ensure both DataFrames can be merged correctly
baseline_df = baseline_df.rename(columns={"lot": "SystemCodeNumber"})

# Merge both DataFrames on 'timestamp' and 'SystemCodeNumber'
# This aligns prices from both models for the same parking lot and time
merged_df = pd.merge(
    baseline_df,
    model3_df,
    on=["timestamp", "SystemCodeNumber"],
    suffixes=("_baseline", "_model3")  # Add suffixes to differentiate columns
)

# Calculate the price difference between the two models
merged_df["price_diff"] = merged_df["price_model3"] - merged_df["price_baseline"]

# Save the merged and annotated output to a new CSV
merged_df.to_csv("model1_vs_model3_comparison.csv", index=False)

# Display a success message and preview of the output
print("Comparison completed. Sample output:")
print(merged_df.head())


# Price Comparison Plot: Model 1 vs Model 2 vs Model 3
This script compares pricing predictions from all three models
for a selected parking lot over time using a line plot.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load model outputs from CSV files
model1_df = pd.read_csv("baseline_output.csv")           # Model 1: Baseline
model2_df = pd.read_csv("model2_demand_output.csv")      # Model 2: Demand-based
model3_df = pd.read_csv("model3_output.csv")             # Model 3: Geo + Demand

# Rename 'lot' to 'SystemCodeNumber' in Model 1 and 2 for consistent merging
for df in [model1_df, model2_df]:
    if 'lot' in df.columns:
        df.rename(columns={'lot': 'SystemCodeNumber'}, inplace=True)

# Convert timestamp strings to datetime objects for plotting
for df in [model1_df, model2_df, model3_df]:
    df['timestamp'] = pd.to_datetime(df['timestamp'])

# Rename price columns to distinguish between models
model1_df.rename(columns={"price": "price_model1"}, inplace=True)
model2_df.rename(columns={"price": "price_model2"}, inplace=True)
model3_df.rename(columns={"price": "price_model3"}, inplace=True)

# Merge all three model outputs on timestamp and lot number
merged_df = model1_df.merge(model2_df, on=["timestamp", "SystemCodeNumber"], how="inner")
merged_df = merged_df.merge(model3_df, on=["timestamp", "SystemCodeNumber"], how="inner")

# Focus on a specific parking lot to reduce clutter in the plot
lot = "BHMBCCMKT01"
filtered = merged_df[merged_df["SystemCodeNumber"] == lot].sort_values("timestamp")

# --- Plotting Section ---

# Set up figure size
plt.figure(figsize=(14, 6))

# Plot price lines for all 3 models with distinct styles
plt.plot(filtered["timestamp"], filtered["price_model1"], label="Model 1 (Baseline)", linestyle="--", color="blue", marker='o')
plt.plot(filtered["timestamp"], filtered["price_model2"], label="Model 2 (Demand-Based)", linestyle="-.", color="orange", marker='o')
plt.plot(filtered["timestamp"], filtered["price_model3"], label="Model 3 (Geo + Demand)", linestyle="-", color="green", marker='o')

# Format the x-axis with date labels
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
plt.xticks(rotation=45)

# Add axis labels and title
plt.xlabel("Timestamp")
plt.ylabel("Price ($)")
plt.title(f"Price Comparison for Lot: {lot}")

# Enable grid and legend for clarity
plt.grid(True)
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()
